# <span style="color:Green"> Asteroid 2024 YR4 Tracking  </span>

**SBDB**- Small Body Database
**Sentry** -Earth Impact Monitoring System

In [71]:
#!pip install rebound
#!pip install --upgrade certifi requests rebound

   ---------------------------------------- 0.0/166.4 kB ? eta -:--:--
   ------- -------------------------------- 30.7/166.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 166.4/166.4 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.8.30
    Uninstalling certifi-2024.8.30:
      Successfully uninstalled certifi-2024.8.30
  Attempting uninstall: requests
    Found existing installation: requests 2.32.2
    Uninstalling requests-2.32.2:
      Successfully uninstalled requests-2.32.2


In [73]:
import requests
import pandas as pd
import numpy as np
import rebound


In [ ]:
context = ssl._create_unverified_context()
urllib.request.urlopen = lambda url, context=context: urllib.request.urlopen(url, context=context)

In [74]:
#Pulling data from SDBD
sbdb_url = "https://ssd-api.jpl.nasa.gov/sbdb.api?sstr=2024%20YR4"
response = requests.get(sbdb_url)
data = response.json()
sbdb_df = pd.DataFrame(data["orbit"]["elements"])
sbdb_df.to_csv("2024_YR4_SBDB_Data.csv", mode="a", header=not pd.io.common.file_exists("2024_YR4_SBDB_Data.csv"))

In [75]:
sbdb_df.head()

,units,sigma,title,label,value,name
0,None,3.6e-06,eccentricity,e,0.662,e
1,au,2.5e-05,semi-major axis,a,2.52,a
2,au,5.3e-07,perihelion distance,q,0.851,q
3,deg,1.3e-05,inclination; angle with respect to x-y eclipti...,i,3.41,i
4,deg,9.2e-06,longitude of the ascending node,node,271,om


In [76]:
print(sbdb_df[["name","value","sigma"]])

   name        value    sigma
0     e        0.662  3.6e-06
1     a         2.52  2.5e-05
2     q        0.851  5.3e-07
3     i         3.41  1.3e-05
4    om          271  9.2e-06
5     w          134  1.6e-05
6    ma         40.4  0.00062
7    tp  2460636.918    4e-05
8   per         1460    0.022
9     n        0.247  3.7e-06
10   ad         4.18  4.2e-05


In [77]:
# Pullig data from Sentry data base
sentry_url = "https://ssd-api.jpl.nasa.gov/sentry.api?des=2024%20YR4"
response = requests.get(sentry_url)
data_sentry = response.json()
sentry_df = pd.DataFrame([data_sentry["summary"]])
sentry_df.to_csv("2024_YR4_sentry_Data.csv", mode="a", header=not pd.io.common.file_exists("2024_YR4_sentry_Data.csv"))

In [78]:
sentry_df.head()

,ps_cum,method,ndel,ts_max,pdate,nobs,darc,first_obs,v_imp,ps_max,...,v_inf,cdate,h,ndop,last_obs,n_imp,energy,diameter,nsat,des
0,-1.11,IOBS,0,1,2025-02-23 16:37:36,404,58.929 days,2024-12-25,17.32,-1.11,...,13.26,2025-02-22 08:20:57,23.96,0,2025-02-22,5,7.828e+00,0.055,0,2024 YR4


## Running simulation

In [90]:
# Using rebound with SBDB elements to propagate to Sentry’s 2032-12-22.59 date.


# Assuming sbdb_df is loaded from your earlier SBDB script
e = float(sbdb_df[sbdb_df["name"] == "e"]["value"].iloc[0])      # 0.662
a = float(sbdb_df[sbdb_df["name"] == "a"]["value"].iloc[0])      # 2.52 AU
i = float(sbdb_df[sbdb_df["name"] == "i"]["value"].iloc[0])      # 3.41 deg
om = float(sbdb_df[sbdb_df["name"] == "om"]["value"].iloc[0])    # 271 deg
w = float(sbdb_df[sbdb_df["name"] == "w"]["value"].iloc[0])      # 134 deg
ma = float(sbdb_df[sbdb_df["name"] == "ma"]["value"].iloc[0])    # 40.4 deg

# Convert angles to radians
i_rad = np.radians(i)
om_rad = np.radians(om)
w_rad = np.radians(w)
ma_rad = np.radians(ma)

# Set up simulation
sim = rebound.Simulation()
sim.units = ('AU', 'day', 'Msun')
sim.add(m=1.0, hash="Sun")  # Sun, mass = 1 solar mass

# Add Earth manually (approximate elements for JD 2460800.5)
sim.add(m=3.0e-6, a=1.000, e=0.0167, inc=np.radians(0.0), Omega=np.radians(174.9), omega=np.radians(288.1), M=np.radians(65.0), hash="Earth")
# Earth: mass=3.0e-6 Msun, a=1 AU, e=0.0167, i=0°, node=174.9°, peri=288.1°, M adjusted for epoch

# Add 2024 YR4
sim.add(a=a, e=e, inc=i_rad, Omega=om_rad, omega=w_rad, M=ma_rad, hash="2024 YR4")

# Debug
print(f"Number of particles: {len(sim.particles)}")  # Should be 3

# Integrate
sim.dt = 0.01  # Small timestep for stability
sim.integrate(2463602.59 - 2460800.5)  # 2032-12-22.59 - 2025-03-06
print(f"2024 YR4 position: {sim.particles['2024 YR4'].xyz}")

Number of particles: 3
2024 YR4 position: [-0.2865124528042221, 1.0288183692925688, -0.01600053555908072]


**Meaning of Output**

* Number of particles: 3: Confirms Sun (index 0), Earth (index 1), and 2024 YR4 (index 2) are in the simulation—perfect.
* 2024 YR4 position: [-0.2865124528042221, 1.0288183692925688, -0.01600053555908072] AU


This is the heliocentric position (x, y, z) in AU, in the J2000 ecliptic frame, on JD 2463602.59 (December 22, 2032, 14:10 UTC).

Step 1: Interpret 2024 YR4’s Position
Coordinates:
x = -0.2865 AU
y = 1.0288 AU
z = -0.0160 AU

In [100]:
# Distance from Sun
r = (x^2 + y^2 + z^2)^0.5 = ((-0.2865)^2 + (1.0288)^2 + (-0.0160)^2)^0.5 = 1.067
r

SyntaxError: cannot assign to expression (1641683763.py, line 2)

## Compare to Earth’s Position
To see if this aligns with Sentry’s potential impact on 2032-12-22.59 (0.3594% probability), we need Earth’s position at the same time:

Earth in Sim: You added Earth with approximate elements (a=1.0 AU, e=0.0167, inc=0°, Omega=174.9°, omega=288.1°, M=65° at JD 2460800.5).
Earth’s Position: After integrating ~2802.09 days (~7.67 years):
Earth’s period is ~365.25 days, so it completes ~7.67 / 1 ≈ 7.67 orbits, or ~2800° of mean anomaly, wrapping around to ~65° + (0.67 * 360°) ≈ 306° (adjusted for precession and eccentricity).
Approximate position (simplified circular orbit at 1 AU):
x_E = cos(306°) ≈ 0.809
y_E = sin(306°) ≈ -0.588
z_E ≈ 0 (near ecliptic).

In [106]:
# FInding earth's position
print(f"Earth position: {sim.particles['Earth'].xyz}")

Earth position: [0.6597866950331682, 0.7381686233295313, 0.0]


## Distance to Earth
* 2024 YR4: [-0.2865, 1.0288, -0.0160]
* Earth (estimated): [0.809, -0.588, 0.0] (rough, based on M=306°).

In [114]:
dx = 0.809 - (-0.2865) = 1.0955
dy = -0.588 - 1.0288 = -1.6168
dz = 0.0 - (-0.0160) = 0.0160
distance = (dx^2 + dy^2 + dz^2)^0.5 = (1.0955^2 + 1.6168^2 + 0.0160^2)^0.5 = 1.96 AU

SyntaxError: cannot assign to expression (1610978534.py, line 1)